# Product selection problem

**Text:** A company has the capacity to produce two types of products, A and B, using available resources. Product A has a profit margin of 300 CHF per unit, while product B has a profit margin of 240 CHF per unit. Each unit of product A requires 240 units of components, while each unit of product B requires 144 units. Additionally, both products require 60 minutes for assembly.
Given that the company has 24,000 units of components and 16 days of production time, with each day consisting of 8 hours of total productive time, the objective is to determine the optimal quantities of products A and B to produce in order to maximize profit.


In [ ]:
!pip3 install ortools

In [ ]:
from ortools.linear_solver import pywraplp


Objective:
Maximize profit by determining the optimal quantities of products A and B to produce.

Constraints:
1. Limited availability of 24,000 units of components.
2. Limited production time of 16 days, each comprising 8 hours of productive time per day.

Decision Variables:
Let $x$ represent the quantity of product A produced.
Let $y$ represent the quantity of product B produced.

Mathematical Formulation:
Maximize: $300x + 240y$ (Total profit)

Subject to:
1. $240x + 144y \leq 24,000$ (Component availability constraint)
2. $60x + 60y \leq 16 \cdot 8 \cdot 60$ (Production time constraint)

Where:
- $x \geq 0$ (Non-negativity constraint for product A)
- $y \geq 0$ (Non-negativity constraint for product B)

The goal is to find the values of $x$ and $y$ that satisfy the constraints and maximize the total profit.

In [ ]:
# Solver
solver = pywraplp.Solver.CreateSolver('CBC')

# Variables
x = solver.IntVar(0, solver.infinity(), 'x')
y = solver.IntVar(0, solver.infinity(), 'y')

In [ ]:
solver.Add(240*x + 144*y <= 24000)
solver.Add(60*x + 60*y <= 7680)

solver.Maximize (300*x + 240*y)

# Solve the system.
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('A =', x.solution_value())
    print('B =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

Solution:
Objective value = 34200.0
A = 58.0
B = 70.0


## Sudoku exercise solution

In [ ]:
from ortools.sat.python import cp_model


def solve_sudoku():
    """Solves the sudoku problem with the CP-SAT solver."""
    # Create the model.
    model = cp_model.CpModel()

    cell_size = 3
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    initial_grid = [[0, 6, 0, 0, 5, 0, 0, 2, 0], [0, 0, 0, 3, 0, 0, 0, 9, 0],
                    [7, 0, 0, 6, 0, 0, 0, 1, 0], [0, 0, 6, 0, 3, 0, 4, 0, 0],
                    [0, 0, 4, 0, 7, 0, 1, 0, 0], [0, 0, 5, 0, 9, 0, 8, 0, 0],
                    [0, 4, 0, 0, 0, 1, 0, 0, 6], [0, 3, 0, 0, 0, 8, 0, 0, 0],
                    [0, 2, 0, 0, 4, 0, 0, 5, 0]]

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.NewIntVar(1, line_size, 'grid %i %i' % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.AddAllDifferent(grid[(i, j)] for j in line)

    # AllDifferent on columns.
    for j in line:
        model.AddAllDifferent(grid[(i, j)] for i in line)

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di,
                                          j * cell_size + dj)])

            model.AddAllDifferent(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.Add(grid[(i, j)] == initial_grid[i][j])

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        for i in line:
            print([int(solver.Value(grid[(i, j)])) for j in line])


solve_sudoku()

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]
